In [11]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool

In [12]:
load_dotenv()
client = AzureOpenAIChatCompletionClient(
    model=os.environ["AZURE_OPENAI_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
)

In [13]:
result = await client.create(
    [UserMessage(content="What is the capital of France?", source="user")]
)
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=8) cached=False logprobs=None thought=None


Defining the Function tool

In [14]:
def vacation_destination(city: str) -> tuple[str, str]:
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Available",
        "Hanoi": "Available"
    }
    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"
    
get_vacation = FunctionTool(
    func=vacation_destination,
    description="Search for vacation destinations and if they are available or not"
)

Defining the agent

In [23]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacation],
    reflect_on_tool_use=True,
    description="You are a travel agent that helps users find vacation destinations"
)
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(
            content="I would like to take a trip to Barcelona",
            source="user"
        )],
        cancellation_token=CancellationToken()
    )
    # print(response.inner_messages)
    # print(response.chat_message)
    print(response.chat_message.content)
    
await assistant_run()

Your trip to Barcelona is available! If you need help with planning, such as accommodations, activities, or travel tips, just let me know. 

TERMINATE
